# 学習データの作成

Twitterデータから学習データを作成する。

事前に準備が必要なもの

- 一行1ツイートを表すJSONデータからなる tweet.json

In [ ]:
tweet_file = "tweets_10000.json"
out_dir = "output"
test_valid_size_per_emoji = 2

In [ ]:
! head tweets.json | jq -c '.text'

In [ ]:
! pip install emoji==0.5.4 pandas==1.0.1 matplotlib==3.2.0

In [ ]:
import matplotlib
%matplotlib inline

## 前処理

tweets.json一行一行から、学習データとして使うツイートをフィルターし、その後前処理をかける

フィルターの条件
- 文字数が100より大きい
- URLが含まれている
- RT


In [ ]:
import json
import re


def filter_tweet(text):
    return any([
        len(text) > 100,
        "http" in text,
        "RT" in text,
        re.search("^[@].*@", text),
        re.search("^\s*$", text)
    ])


def preprocess_tweet(text):
    text = re.sub("^@[0-9a-zA-Z_]+\s*", "", text)
    text = re.sub("[\n\t\s]", "", text)
    return text


def filter_and_preprocess(fd):
    texts = []
    num_total = 0
    for line in fd:
        num_total += 1
        tweet_json = line.strip("\n")
        text = json.loads(tweet_json)["text"]
        if filter_tweet(text):
            continue
        preprocessed_text = preprocess_tweet(text)
        texts.append(preprocessed_text)
    print("Number of texts before filtering:", num_total)
    print("Number of texts after filtering:", len(texts))

    return texts

In [ ]:
texts = filter_and_preprocess(open(tweet_file))

In [ ]:
[print(x) for x in texts[:10]]

前処理を行ったデータから、最終的な学習コーパスを作成する。

文末に絵文字がつく文を抽出し、その絵文字逹を文への絵文字として扱う。
複数絵文字が付与されている場合は、一つ一つがその文への絵文字であるとする。

In [ ]:
import emoji
import re


class EmojiExtractor:
    def __init__(self):
        self._regex = "(" + "|".join(emoji.EMOJI_UNICODE) + ")"

    def extract(self, text):
        edict = dict()
        detext = emoji.demojize(text)
        last_match = 0
        last_text = ""
        for match in re.finditer(self._regex, detext):
            emj = match.group()
            etxt = detext[last_match:match.start()]
            last_tmp = last_match
            last_match = match.end()
            if match.start() == last_tmp:
                if not last_text:
                    continue
                etxt = last_text
            else:
                last_text = etxt
            edict[etxt] = edict.get(etxt, set()) | {emj}
        return edict

In [ ]:
from collections import Counter


def make_corpus(texts):
    emoji_extractor = EmojiExtractor()
    dataset = []
    emoji_num_counter = Counter()
    for text in texts:
        text_emoji_dict = emoji_extractor.extract(text)
        for sent, emoji_set in text_emoji_dict.items():
            emoji_num_counter.update([len(emoji_set)])
            for emoji_ in emoji_set:
                # 学習に利用する絵文字であれば学習データに追加
                #if emoji_ in emoji_id:
                dataset.append((emoji_, sent))
    print("Number of emojis distribution per sentence:", emoji_num_counter)
    return dataset
        
        
dataset = make_corpus(texts)

In [ ]:
import pandas as pd


df = pd.DataFrame(dataset, columns=["emoji", "text"])

## データセットの分析

In [ ]:
import matplotlib
import matplotlib.ticker as mtick

dataset_size = len(df)
print("Dataset size: ", dataset_size)

emoji_count = df.emoji.value_counts()
emoji_cumcount = emoji_count.cumsum()

# ラベル数を調べる
print("Unique labels:", len(emoji_count))

# 頻度情報を付与したデータフレームを作成
emoji_df = pd.DataFrame({
    "emoji": [emoji.EMOJI_UNICODE[x] for x in emoji_count.index],
    "count": emoji_count,
    "perc": emoji_count * 100 / dataset_size,
    "cum_count": emoji_cumcount,
    "cum_perc": emoji_cumcount *100 / dataset_size,
})

emoji_df

In [ ]:
# 全体の80%までの絵文字に制限する
selected_emoji_df = emoji_df[emoji_df.cum_perc <= 80]
print("Dataset size after selection:", selected_emoji_df.cum_count[-1])
print("Unique labels after selection:", len(selected_emoji_df))

In [ ]:
selected_emoji_df

In [ ]:
selected_emoji_df.perc.plot(kind="bar", figsize=(20, 5))

## 学習データの作成

絵文字を制限したデータセットから学習データを作成する。
作成する際に注意することは

- 不均衡データなので、ダウンサンプリングして均衡データとして扱う

In [ ]:
selected_emoji_names = set(selected_emoji_df.index)
selected_df = df[[x in selected_emoji_names for x in df.emoji]]
emoji_grouped_df = selected_df.groupby("emoji")

# Check if the dataset is sufficient to split train, valid and test set
num_min_emoji = emoji_grouped_df.size().min()
assert num_min_emoji >= test_valid_size_per_emoji * 3

split_dataset = []
from_to_list = [
    (0, test_valid_size_per_emoji),
    (test_valid_size_per_emoji, test_valid_size_per_emoji * 2), 
    (test_valid_size_per_emoji * 2, num_min_emoji)

]

for (from_, to_) in from_to_list:
    grouped_dataset = selected_df.groupby("emoji").nth(list(range(from_, to_)))
    split_dataset.append(grouped_dataset)
    
test_set, valid_set, train_set = split_dataset
print("Dataset size, train: {}, valid: {}, test: {}".format(len(train_set), len(valid_set), len(test_set)))

学習データを保存する

In [ ]:
for (target_df, filename, index) in ([(selected_df, "all.tsv", False), (train_set, "train.tsv", True), (valid_set, "valid.tsv", True), (test_set, "test.tsv", True)]):
    target_df.to_csv(out_dir+ "/" + filename, sep="\t", header=None, index=index)